In [1]:
import open3d as otd
from tqdm import tqdm
import torch as tc
import numpy as np
import pandas as pd
from curve_utils import CurveUtils
from curve_generator import CurveGenerator
from bbox import BBox
from mesh_manipulation import save_obj

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

In [2]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Total crotch length',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'stature', # 5.1.1
    'crotch_length', # 5.1.15
    'shoulder_width', # 5.4.3
]
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    'stature': 0,
    'crotch_length': 5
}
curves_names = list(curve_index.keys())
body_files = {
    "female": "tests/female_4_body.obj",
    "male": "tests/male_2_body.obj"
}

In [3]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    measures.index = measures['Subject']
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures._get_numeric_data().mean(axis=0)
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [7]:
for gender in ['male', 'female']:
    mesh = otd.io.read_triangle_mesh(body_files[gender])
    for nsubs in range(3):
        temp_mesh = mesh.subdivide_loop(number_of_iterations=nsubs)
        print(f'Gender:{gender} - Mesh:{np.array(temp_mesh.triangles).shape[0]} triangles')
        body = tc.FloatTensor(np.asarray(temp_mesh.vertices)).to(device)
        template = tc.LongTensor(np.asarray(temp_mesh.triangles)).to(device)
        measures = selected_measures[gender]
        result = CurveGenerator.get_curves(body, measures, template, device)
        best_curves, data = CurveUtils.select_better(result, body, measures, device)
        data['measure'] = curves_names[:-1]
        data = data.set_index('measure')
        print(data[:-1])

        # all_positions = []
        # for curve in best_curves:
        #     position = CurveUtils.generate_positions(curve, body)
        #     position, measures = CurveGenerator.ray_polygon_colision(body[template], position)
        #     all_positions.extend(position)
        # all_positions = tc.row_stack(all_positions)
        # save_mesh = otd.geometry.TriangleMesh()
        # save_mesh.vertices = otd.utility.Vector3dVector(all_positions.cpu().numpy())
        # otd.io.write_triangle_mesh("teste.obj", save_mesh)


Gender:male - Mesh:13776 triangles
                    original    measured  error(mm)
measure                                            
neck_girth         40.458599   40.455730   0.028687
bust_chest_girth  110.261002  110.250946   0.100555
waist_girth        96.418800   96.417961   0.008392
hip_girth         108.165001  108.077812   0.871887
upper_arm_girth    33.042099   33.026119   0.159798
thigh_girth        64.755295   64.771461   0.161667
Gender:male - Mesh:55104 triangles
                    original    measured  error(mm)
measure                                            
neck_girth         40.458599   40.458843   0.002441
bust_chest_girth  110.261002  110.277031   0.160294
waist_girth        96.418800   96.424309   0.055084
hip_girth         108.165001  108.139404   0.255966
upper_arm_girth    33.042099   33.112026   0.699272
thigh_girth        64.755295   64.793015   0.377197
Gender:male - Mesh:220416 triangles
                    original    measured  error(mm)
measure   

In [ ]:
selected_body = tc.FloatTensor(np.asarray(mesh.vertices)).to(device)
template = tc.LongTensor(np.asarray(mesh.triangles)).to(device)
body_min = selected_body[:,1].min()
width = selected_body[:,0].max() - selected_body[:,0].min()
height = selected_measures['female']['stature']/1000
center = (0, body_min + (height/2), 0)
box = BBox(width=width, height=height, center=tc.FloatTensor(center).to(device))
coor, meas, posi = CurveGenerator.calculate_curve(selected_body, box, 1, template, device)